In [1]:
from scanr_utils import *
from casuhal_utils import *
import pandas as pd, json

## 1. Récupérer les DOI

In [2]:

## option a : utilsier un fichier local 
utiliser_fichier_local = True # True pour oui False pour non

if utiliser_fichier_local : 
    ## depuis un fichier local placé dans le dossier *data*
    ## doit être un fichier .csv avec encodage utf8 et ',' comme séparateur
    ## avec au moins une colonne *doi*
    df_mydois = pd.read_csv("data\scopus_2021.csv", usecols=["doi"])     
    print(f"nb de DOI importés depuis fichier \t{len(df_mydois)}")
else : 
    df_mydois = pd.DataFrame()

nb de DOI importés depuis fichier 	1283


In [3]:
## option b : utiliser ScanR pour récupérer les DOIs

utiliser_scanR = False # True pour oui False pour non 
# scanR inclut les publications jusqu'a 2020.
# à UP les DOIs récents sont priorisé, donc dans un premier temps pas d'utilisation de ScanR

if utiliser_scanR : 

    id_etab = "0755976N" # identifiant UAI de votre établissement
    min_year = 2020 # l'année à partir de laquelle commencer la récupération. 
    # 2020 renvoie les publications de 2020. 2019 renvoie les publications de 2019 et 2020 etc.

    df_scanr = get_publications_with_doi(id_etab, min_year)
    df_scanr.drop(columns = ["title", "year"], inplace = True)
    print(f"nb de DOI récupérés de ScanR \t{len(df_scanr)}")
    #df_scanr.to_csv("data\doi_scanr.csv", columns= ["doi"], index = False)
    
    # concatener et dédoublonner
    df = pd.concat([df_mydois, df_scanr])
    df["doi"] = df["doi"].str.lower() # doi en minuscule
    df = df[ (~df['doi'].duplicated()) & (df["doi"].notna())].copy()
    df.reset_index(drop=True, inplace=True)
    
else : 
    df = df_mydois

In [4]:
print(f" nb de DOI a traiter \t{len(df)}")
print(f" /!\ temps estimé ~{round(len(df) * 30/1200)} minutes")

 nb de DOI a traiter 	1283
 /!\ temps estimé ~32 minutes


## 2. Enrichir

In [ ]:
## avec HAL
df_hal = enrich_w_hal(df) # renseigner df[:50].copy() pour tester uniquement sur les 50 premiers DOI
df_no_file = df_hal[ df_hal["submitType"] != "file" ].copy()
print(f"nb de DOI après retrait de ceux en TI Dans HAL {len(df_no_file)}")

In [ ]:
## avec unpaywall
df_upw = enrich_w_upw(df_no_file)

In [ ]:
## avec Permissions
df_permissions = enrich_w_permissions(df_upw)

In [ ]:
## export des données brutes
df_permissions.to_csv("data\dois_hal_upw_perm.csv", index = False)

## 3. Trier et Formater

In [ ]:
df_final = df_permissions
df_final.fillna("", inplace=True)# sinon nan sera compris comme une string non vide

## retirer ce qui est dans HAL qui a un lien extérieur et qui est signalé en repository dans upw
index2remove = df_final[ (df_final["linkExtId"] != "") & (df_final["oa_repo_link"] != "") ].index
df_final.drop(index2remove, inplace = True)
print(f"nb DOI retiré car marqué 'open' dans HAL et repository dans upw {len(index2remove)}" )

In [ ]:
#deduire la colonne todo
df_final["todo"] = df_final.apply(lambda row : deduce_todo(row), axis = 1)
df_final.to_csv("data\dois_a_traiter.csv", index = False)
#imprimer des statistiques brutes
print(f"\n\nStatistiques\n\nnb de DOI a traiter\t\t{len(df_final)}\n{df_final['todo'].value_counts()}")

In [ ]:
#retrait des colonnes inutiles 
df_final.drop(columns= ["submitType", "has_issn"], inplace = True)

# rendre la lecture dans libreOffice plus élégante avec l'ajout d'hyperliens
df_final["doi"] = df_final.apply(lambda row : addCaclLinkFormula("https://doi.org/", row["doi"], row["doi"]), axis = 1)  
df_final["halId"] = df_final.apply(lambda row : addCaclLinkFormula("https://hal.archives-ouvertes.fr/", row["halId"], row["halId"]), axis = 1)  
df_final["oa_publisher_link"] = df_final.apply(lambda row : addCaclLinkFormula("", row["oa_publisher_link"], row["oa_publisher_link"]), axis = 1)	
df_final["oa_repo_link"] = df_final.apply(lambda row : addCaclLinkFormula("", row["oa_repo_link"], row["oa_repo_link"]), axis = 1)	

df_final.to_csv("data\dois_a_traiter_formules.csv", index = False)